<a href="https://colab.research.google.com/github/ardabyr/DataScience/blob/main/%D0%9C%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8_%D0%B2_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# ПРАКТИЧЕСКАЯ ЧАСТЬ: МЕТРИКИ К РЕАЛИЗАЦИИ

## 1. ОПИСАНИЕ МЕТРИК

Качество работы рекомендательных систем определяется релевантностью рекомендаций, для оценки которой существует ряд метрик. Релевантность определяется как достижение целевого взаимодействия (пользователь совершил целевое действие с рекомендованным элементом, т.е. перешел по ссылке, просмотрел фильм, оставил отзыв и т.д.).

Ниже будут кратко рассмотрены следующие из часто используемых семейств метрик, которые вам необходимо будет реализовать в ходе практической работы:
-	**Hit Rate** (включая Hit Rate@k);
-	**Precision** (включая Precision@k, MoneyPrecision@k и (Mean)AveragePrecision@k);
-	**Recall** (включая Recall@k, MoneyRecall@k и (Mean)AverageRecall@k);
-	**MRR** (Mean Reciprocal Rank, включая MRR@k);
-	**DCG** (Discounted Cumulative Gain, включая DCG@k, nDCG и nDCG@k).

=====

**NB!** *«@k» в названии метрики означает, что метрика измеряется на первых k рекомендациях; например, если k = 5, то «попадание» засчитывается как успешное, только если оно входит в первые 5 рекомендаций. «Mean» означает, что измеряется среднее значение по всей выборке.*

**Сущность рассматриваемых метрик:**

**Hit Rate**: определяет, удалось ли в рекомендациях верно подобрать хотя бы один релевантный для пользователя элемент. Hit Rate@k, соответственно, оценивает тот же показатель, но исключительно в пределах первых k рекомендаций.

**Precision**: вычисляет, какую долю рекомендаций пользователь фактически счел релевантными (совершил целевое действие). Т.е. сколько, например, товаров он купил из тех, что мы ему предложили? Precision@k измеряет эту долю в топ k. MoneyPrecision@k аналогична Precision@k, но учитывает стоимость рекомендуемых элементов; (Mean)AveragePrecision@k — более комплексная метрика, которая учитывает порядок рекомендаций и присваивает более высокие баллы системам, рекомендующим релевантные элементы в начале списка.

**Recall**: вычисляет, какая доля из элементов, сочтенных пользователем релевантными, была ему фактически рекомендована. Т.е. сколько из реально купленного (например) мы пользователю порекомендовали. Recall@k, MoneyRecall@k и (Mean)AverageRecall@k — аналогично семейству Precision.

**MRR** (Mean Reciprocal Rank): измеряет средний ранг первого релевантного элемента в общем списке рекомендаций, MRR@k — в k лучших рекомендациях.

**DCG** (Discounted Cumulative Gain): измеряет качество ранжирования рекомендуемых элементов. DCG@k измеряет качество k лучших рекомендаций, а nDCG@k нормализует оценку DCG для учета количества релевантных элементов.

===

 **NB!** *Отдельно остановимся на MoneyPrecision@k и MoneyRecall@k: они отличаются от всех прочих рассматриваемых метрик тем, что привязываются не только к качеству работы алгоритма как такового, но и к реальной стоимости элементов. Иначе говоря, эти метрики относятся не к чистым ML-метрикам, но к т.н. «бизнес-метрикам», т.е. позволяют оценить непосредственный экономический эффект для предприятия от использования той или иной модели. Расчет ML-метрик и бизнес-метрик, как правило, различается: далеко не всегда использование модели с идеальными ML-показателями экономически оправдано, поскольку бюджеты предприятия ограничены, а их расходование должно окупаться.*


## 2. ПОДГОТОВКА НАБОРА ДАННЫХ

In [ ]:
import numpy as np
import pandas as pd
import random as rn
from math import log, log2

Работу нужно выполнить как с фиксированным датасетом (просто и наглядно), так и с более крупным рандомным. Датасеты имеют одинаковые названия и структуру, что  не отразится на выполнении кода.

Чтобы не усложнять дополнительно задачу и сделать ее более наглядной, дублирование позиций (например, пользователь купил один товар несколько раз) в обоих датасетах исключено. Пользователь также может получить несколько одинаковых рекомендаций, но такая ситуация -- признак недоработки алгоритма, и здесь также не рассматривается.

### А. Фиксированный датасет

In [ ]:
# создадим датафрейм users_rec_bgt_db (сокр. от users_recommended_bought_database),
#  содержащий столбцы:
# (а) user_id: три пользователя -- u1, u2 и u3;
# (б) recommended_list: три списка рекомендаций (номера товаров);
# (в) bought_list: три списка фактических покупок (номера товаров)

users_rec_bgt_db = pd.DataFrame({
    "user_id": ["u1","u2","u3"],
    "recommended_list": [
        [143, 1576, 1134, 991, 27],
        [1543, 3345, 533, 11, 15],
        [156, 3345, 10, 15, 1234]
        ],
    "bought_list": [
        [156,27],
        [11,43],
        [1]
        ]})

users_rec_bgt_db

,user_id,recommended_list,bought_list
0,u1,"[143, 1576, 1134, 991, 27]","[156, 27]"
1,u2,"[1543, 3345, 533, 11, 15]","[11, 43]"
2,u3,"[156, 3345, 10, 15, 1234]",[1]


Первый пользователь (u1) приобрел два товара из пяти рекомендованных: 2-ю и 5-ю позиции.

Второй пользователь (u2) приобрел один товар из рекомендованных (4-ю позицию) и один товар, который не был ему рекомендован (т.е. не попал в рекомендации).

Третий пользователь (u3) приобрел один товар, который не был ему рекомендован, и не приобрел ничего из рекомендованных товаров.

In [ ]:
# создадим датафрейм items_db (сокр. от items_database), содержащий столбцы:
# (а) item_id: идентификаторы товаров;
# (б) item_price: цены на товары

# берем все товары, рекомендованные пользователям, и сортируем
items_list = [143, 1576, 1134, 991, 27, 1543, 3345, 533, 11, 43, 156, 10, 15, 1234, 1]
items_list.sort()

items_db = pd.DataFrame({
    "item_id": items_list,
    "item_price": [1000, 1200, 1400, 1600, 1800,
                   10000, 12000, 14000, 16000, 18000,
                   100000, 120000, 140000, 160000, 180000]
    })

items_db

,item_id,item_price
0,1,1000
1,10,1200
2,11,1400
3,15,1600
4,27,1800
5,43,10000
6,143,12000
7,156,14000
8,533,16000
9,991,18000


По итогам подготовки набора данных вы получили два датафрейма, соответствующие двум таблицам в базе данных:

1. Пользователи + рекомендации + покупки (релевантные позиции). Этот датафрейм будет использоваться для расчета всех метрик.

2. Товары + цены. Этот датафрейм будет использоваться исключительно для расчета бизнес-метрик (MoneyPrecision и MoneyRecall).

## 3. РЕАЛИЗАЦИЯ МЕТРИК

Данный раздел включает в себя 16 метрик:

**Hit Rate**:
1-2) hit_rate, hit_rate_k;

**Precision**:
3-6) precision, precision_k, money_precision_k, average_precision_k;

**Recall**:
7-10) recall, recall_k, money_recall_k, average_recall_k;

**MRR**:
11-12) mrr, mrr_k;

**DCG/nDCG**:
13-16) dcg, ndcg, dcg_k, ndcg_k.

Часть из них уже реализована, оставшуюся часть вам нужно реализовать самостоятельно. Допускается как вызывать функцию из другой функции (например, реализуя метрику recall@k, обращаться к внешней функции recall и т.п.), так и делать их взаимонезависимыми.

В конце раздела содержится код для вывода всех метрик по любому пользователю.

### 3.1. Hit Rate +

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника)

----
Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [ ]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)
    hit_rate = int(flags.sum() > 0)

    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    hit_rate_at_k = hit_rate(recommended_list, bought_list)

    return hit_rate_at_k

### 3.2. Precision +

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precision@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precision@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

**AP@k** - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r = кол-во релевантных среди рекомендованных
- Суммируем по всем релевантным товарам
- Метрика чувствительна к порядку рекомендаций

**MAP@k**

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
*|U|* - кол-во юзеров

**Основные метрики семейства**: Precision, Precision@k, Money_Precision@k (бизнес-метрика), Average_Precision@k

In [ ]:
def precision(recommended_list, bought_list):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)
  precision = 0 if sum(flags) == 0 else sum(flags) / len(recommended_list)

  return precision

def precision_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]
  precision_at_k = precision(recommended_list, bought_list)

  return precision_at_k

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]
  prices_recommended = np.array(prices_recommended)[:k]

  flags = np.isin(recommended_list, bought_list)
  money_precision_at_k = 0 if sum(flags) == 0 else sum(flags * prices_recommended) / sum(prices_recommended)

  return money_precision_at_k

def average_precision_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)
  if sum(flags) == 0:
      return 0
  sum_ = 0

  list_len = min(len(recommended_list), k)
  for i in range(list_len):
      if flags[i]:
          p_k = precision_at_k(recommended_list, bought_list, k=i+1)
          sum_ += p_k
  average_precision_at_k = sum_ / k

  return average_precision_at_k

### 3.3. Recall +

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$

**Метрики семейства**: Recall, Recall@k, Money_Recall@k (бизнес-метрика), Average_Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

---

**Note:** в recall@k число k обычно достаточно большое (50-200) -- больше, чем покупок у среднестатистического юзера

In [ ]:
# доля фактически купленных товаров, показанная в рекомендациях (какой процент из купленных был показан?)
def recall(recommended_list, bought_list):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)
  recall = 0 if sum(flags) == 0 else sum(flags) / len(bought_list)

  return recall

# доля фактически купленных товаров, показанная в рекомендациях (первые k позиций)
def recall_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list[:k], bought_list)
  recall_at_k = 0 if sum(flags) == 0 else sum(flags) / len(bought_list)

  return recall_at_k

# доля стоимости фактически купленных товаров относительно стоимости товаров, показанных в рекомендациях
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)
  prices_recommended = np.array(prices_recommended)
  prices_bought = np.array(prices_bought)

  recommended_prices = prices_recommended[:k]
  bought_prices = prices_bought[np.isin(bought_list, recommended_list[:k])]

  money_recall_at_k = 0 if sum(bought_prices) == 0 else sum(bought_prices) / sum(recommended_prices)


  return money_recall_at_k

def average_recall_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  recalls = []
  for user_recommended_list, user_bought_list in zip(recommended_list, bought_list):
    recalls.append(recall_at_k(user_recommended_list, user_bought_list, k))

  average_recall_at_k = np.mean(recalls)

  return average_recall_at_k

### 3.4. Метрики ранжирования

#### 3.4.1. MRR@k

MRR = Mean Reciprocal Rank:

- Найти ранг (место по порядку) первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

- Считаем для первых k рекомендаций:
$$\Large  MRR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [ ]:
# для всех
def mrr(recommended_list, bought_list):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)

  ranks = np.where(flags)[0] + 1  # индексы, где True
  reciprocal_ranks = 1.0 / ranks
  mrr = np.mean(reciprocal_ranks) if len(ranks) > 0 else 0.0

  return mrr

# для первых k позиций
def mrr_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list[:k], bought_list)
  ranks = np.where(flags)[0] + 1  # индексы, где True
  reciprocal_ranks = 1.0 / ranks
  mrr_at_k = np.mean(reciprocal_ranks) if len(ranks) > 0 else 0.0

  return mrr_at_k

#### 3.4.2. DCG +

DCG = Discounted Cumulative Gain.

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров

**nDCG = Normalized DCG**

$$NDCG = \frac{DCG}{ideal DCG}$$




**DCG@k, nDCG(k)**

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$ и рекомендаций длины $K$. Это значение считаем идеальным DCG (IDCG/idealDCG, > IDCG@k).
Максимальный $DCG$ -- это когда мы порекомендовали максимально возможное количество релевантных продуктов + все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

**!! Внимание!** Данные метрики **dcg/ndcg** реализуются по **иной** логике, нежели те, что доступны в библиотеке sklearn, и возвращают **другие** значения. Они рассчитаны на оценку качества системы в ситуациях, когда список релевантных позиций (покупок и т.д.) может либо не совпадать по длине со списком рекомендаций, либо **отсутствовать вовсе**. Метрики sklearn в таких случаях неприменимы, поскольку требуют на вход два массива одинаковой длины.

In [ ]:
def dcg(recommended_list, bought_list):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)

  if sum(flags) == 0:
    return 0

  rel_ind_array = np.nonzero(flags)[0]
  dcg = sum([1 / log2((j + 1) + 1) for j in rel_ind_array])

  return dcg

def ndcg(recommended_list, bought_list):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)

  if sum(flags) == 0:
    return 0

  rel_ind_array = np.nonzero(flags)[0]
  dcg = sum([1 / log2((j + 1) + 1) for j in rel_ind_array])
  ideal_dcg = sum([1 / log2((j + 1) + 1) for j in range(len(bought_list))])
  ndcg = dcg / ideal_dcg

  return ndcg

def dcg_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)

  flags = np.isin(recommended_list, bought_list)

  if sum(flags) == 0:
    return 0

  rel_ind_array = np.nonzero(flags)[0]
  dcg_at_k = sum([1 / log2((j + 1) + 1) for j in rel_ind_array])

  return dcg_at_k

def ndcg_at_k(recommended_list, bought_list, k=5):
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)[:k]

  dcg_at_k = dcg(recommended_list, bought_list)
  ideal_dcg_at_k = sum([1 / log2((j + 1) + 1) for j in range(len(bought_list))])

  ndcg_at_k = dcg_at_k / ideal_dcg_at_k

  return ndcg_at_k

## 4. РАСЧЕТ МЕТРИК ДЛЯ ЗАДАННОГО/СЛУЧАЙНОГО ПОЛЬЗОВАТЕЛЯ

In [ ]:
k = 4 # задать k
users = users_rec_bgt_db['user_id'].nunique()
# id = rn.randint(1, users-1) # для случайного пользователя
id = 1 # запрашиваем метрики по пользователю №1

user_id = 'u' + str(id)

# что рекомендовали пользователю
user_recs = users_rec_bgt_db.recommended_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# что купил пользователь
user_bgts = users_rec_bgt_db.bought_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# цены рекомендованных товаров
prices_list_rec = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_recs]
# цены реально купленных товаров
prices_list_bgt = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_bgts]

# датафрейм
user = pd.DataFrame(
    {
        'user_id': user_id,
        'rec_items': [user_recs],
        'rel_items': [user_bgts],
        'rec_prices': [prices_list_rec],
        'rel_prices': [prices_list_bgt]
    }
)

# столбцы с метриками
# funcs1, cols1 -- ML-метрики без привязки топ-k, где funcs1 -- список функций, а cols1 -- имена столбцов
funcs1 = (hit_rate, precision, recall, mrr, dcg, ndcg)
cols1 = ('hr', 'p', 'r', 'mrr', 'dcg', 'ndcg')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs1, cols1):
  user[col] = user.apply(lambda user: func(user.rec_items, user.rel_items), axis=1)

# funcs2, cols2 -- ML-метрики на топ-k. они считаются отдельно, т.к. при их вызове нужно задать третий аргумент: k
funcs2 = (hit_rate_at_k,
          precision_at_k, average_precision_at_k,
          recall_at_k, average_recall_at_k,
          mrr_at_k, dcg_at_k, ndcg_at_k)
cols2 = ('hr@', 'p@', 'ap@', 'r@', 'ar@', 'mrr@', 'dcg@', 'ndcg@')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs2, cols2):
#   print(func(user.rec_items, user.rel_items, k))
  try:
    print(func(user.rec_items, user.rel_items, k))
    user[col+str(k)] = user.apply(lambda user: func(user.rec_items, user.rel_items, k), axis=1)
  except IndexError:
    user[col+str(k)] = user.apply(lambda user: 0.5, axis=1)

# бизнес-метрики. nfr;t cчитаются отдельно, т.к. при их вызове нужно задать дополнительные аргумент: передать список/списки с ценами
user['mp@'+str(k)] = user.apply(lambda user: money_precision_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    k), axis=1)
user['mr@'+str(k)] = user.apply(lambda user: money_recall_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    user.rel_prices,
    k), axis=1)

# вывод
# user_file
print(user.iloc[0])

0
0
0
0
0.0
0.0
0
0.0
user_id                                         u1
rec_items               [143, 1576, 1134, 991, 27]
rel_items                                [156, 27]
rec_prices    [12000, 160000, 100000, 18000, 1800]
rel_prices                           [14000, 1800]
hr                                               1
p                                              0.2
r                                              0.5
mrr                                            0.2
dcg                                       0.386853
ndcg                                      0.237198
hr@4                                             0
p@4                                              0
ap@4                                           0.0
r@4                                              0
ar@4                                           0.5
mrr@4                                          0.0
dcg@4                                     0.386853
ndcg@4                                         0.0
mp@4     

In [ ]:
k = 4 # задать k
users = users_rec_bgt_db['user_id'].nunique() # количество пользователей
# id = rn.randint(1, users-1) # для случайного пользователя
id = 2 # запрашиваем метрики по пользователю №2

user_id = 'u' + str(id)

# что рекомендовали пользователю
user_recs = users_rec_bgt_db.recommended_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# что купил пользователь
user_bgts = users_rec_bgt_db.bought_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# цены рекомендованных товаров
prices_list_rec = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_recs]
# цены реально купленных товаров
prices_list_bgt = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_bgts]

# датафрейм
user_file = pd.DataFrame(
    {
        'user_id': user_id,
        'rec_items': [user_recs],
        'rel_items': [user_bgts],
        'rec_prices': [prices_list_rec],
        'rel_prices': [prices_list_bgt]
    }
)

# столбцы с метриками
# funcs1, cols1 -- ML-метрики без привязки топ-k, где funcs1 -- список функций, а cols1 -- имена столбцов
funcs1 = (hit_rate, precision, recall, mrr, dcg, ndcg)
cols1 = ('hr', 'p', 'r', 'mrr', 'dcg', 'ndcg')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs1, cols1):
  user_file[col] = user_file.apply(lambda user: func(user.rec_items, user.rel_items), axis=1)

# funcs2, cols2 -- ML-метрики на топ-k. они считаются отдельно, т.к. при их вызове нужно задать третий аргумент: k
funcs2 = (hit_rate_at_k,
          precision_at_k, average_precision_at_k,
          recall_at_k, average_recall_at_k,
          mrr_at_k, dcg_at_k, ndcg_at_k)
cols2 = ('hr@', 'p@', 'ap@', 'r@', 'ar@', 'mrr@', 'dcg@', 'ndcg@')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs2, cols2):
  try:
    print(func(user.rec_items, user.rel_items, k))
    user[col+str(k)] = user.apply(lambda user: func(user.rec_items, user.rel_items, k), axis=1)
  except IndexError:
    user[col+str(k)] = user.apply(lambda user: 0.5, axis=1)

# бизнес-метрики. nfr;t cчитаются отдельно, т.к. при их вызове нужно задать дополнительные аргумент: передать список/списки с ценами
user_file['mp@'+str(k)] = user_file.apply(lambda user: money_precision_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    k), axis=1)
user_file['mr@'+str(k)] = user_file.apply(lambda user: money_recall_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    user.rel_prices,
    k), axis=1)

# вывод
# user_file
print(user_file.iloc[0])

0
0
0
0
0.0
0.0
0
0.0
user_id                                        u2
rec_items               [1543, 3345, 533, 11, 15]
rel_items                                [11, 43]
rec_prices    [140000, 180000, 16000, 1400, 1600]
rel_prices                          [1400, 10000]
hr                                              1
p                                             0.2
r                                             0.5
mrr                                          0.25
dcg                                      0.430677
ndcg                                     0.264068
mp@4                                     0.004149
mr@4                                     0.004149
Name: 0, dtype: object


In [ ]:
k = 5 # задать k
users = users_rec_bgt_db['user_id'].nunique()
# id = rn.randint(1, users-1) # для случайного пользователя
id = 3 # запрашиваем метрики по пользователю №3

user_id = 'u' + str(id)

# что рекомендовали пользователю
user_recs = users_rec_bgt_db.recommended_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# что купил пользователь
user_bgts = users_rec_bgt_db.bought_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# цены рекомендованных товаров
prices_list_rec = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_recs]
# цены реально купленных товаров
prices_list_bgt = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_bgts]

# датафрейм
user_file = pd.DataFrame(
    {
        'user_id': user_id,
        'rec_items': [user_recs],
        'rel_items': [user_bgts],
        'rec_prices': [prices_list_rec],
        'rel_prices': [prices_list_bgt]
    }
)

# столбцы с метриками
# funcs1, cols1 -- ML-метрики без привязки топ-k, где funcs1 -- список функций, а cols1 -- имена столбцов
funcs1 = (hit_rate, precision, recall, mrr, dcg, ndcg)
cols1 = ('hr', 'p', 'r', 'mrr', 'dcg', 'ndcg')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs1, cols1):
  user_file[col] = user_file.apply(lambda user: func(user.rec_items, user.rel_items), axis=1)

# funcs2, cols2 -- ML-метрики на топ-k. они считаются отдельно, т.к. при их вызове нужно задать третий аргумент: k
funcs2 = (hit_rate_at_k,
          precision_at_k, average_precision_at_k,
          recall_at_k, average_recall_at_k,
          mrr_at_k, dcg_at_k, ndcg_at_k)
cols2 = ('hr@', 'p@', 'ap@', 'r@', 'ar@', 'mrr@', 'dcg@', 'ndcg@')

# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs2, cols2):
  try:
    print(func(user.rec_items, user.rel_items, k))
    user[col+str(k)] = user.apply(lambda user: func(user.rec_items, user.rel_items, k), axis=1)
  except IndexError:
    user[col+str(k)] = user.apply(lambda user: 0.5, axis=1)

# бизнес-метрики. nfr;t cчитаются отдельно, т.к. при их вызове нужно задать дополнительные аргумент: передать список/списки с ценами
user_file['mp@'+str(k)] = user_file.apply(lambda user: money_precision_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    k), axis=1)
user_file['mr@'+str(k)] = user_file.apply(lambda user: money_recall_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    user.rel_prices,
    k), axis=1)

# вывод
# user_file
print(user_file.iloc[0])

0
0
0
0
0.5
0.0
0
0.0
user_id                                        u3
rec_items               [156, 3345, 10, 15, 1234]
rel_items                                     [1]
rec_prices    [14000, 180000, 1200, 1600, 120000]
rel_prices                                 [1000]
hr                                              0
p                                               0
r                                               0
mrr                                           0.0
dcg                                             0
ndcg                                            0
mp@5                                            0
mr@5                                            0
Name: 0, dtype: object


### Б. Рандомный датасет

Проведите оценку качества работы рекомендательной системы для датасета, сгенерированного рандомно. Необходимо задать номер варианта ниже.

In [ ]:
var_num = 4 #номер варианта
users = (var_num + 8) % 10 + 3 # задать кол-во пользователей в базе данных
rec_len = var_num % 10+ 3 # задать длину списка рекомендованных товаров/позиций
bgt_len = (0,6) # задать границы списка покупок: кто-то мог купить 6 товаров, а кто-то -- ни одного
item_range = var_num % 10+12 # задать длину диапазона позиций

# !! ситуации с дублированием позиций исключены (т.е. использован sample вместо randint)

users_rec_bgt_db = pd.DataFrame({
    "user_id": ["u"+str(i+1) for i in range(users)],

    "recommended_list": [rn.sample(range(1, item_range), rec_len)
                         for i in range(users)],

    "bought_list": [rn.sample(range(1, item_range), rn.randint(bgt_len[0],
                                                               bgt_len[1]))
                            for i in range(users)]
    })

users_rec_bgt_db

,user_id,recommended_list,bought_list
0,u1,"[11, 1, 8, 10, 6, 3, 9]","[8, 6, 10]"
1,u2,"[6, 1, 11, 7, 9, 4, 3]","[3, 2, 12, 14, 10]"
2,u3,"[10, 9, 1, 13, 7, 4, 5]","[1, 14, 13]"
3,u4,"[1, 11, 9, 8, 10, 14, 2]","[10, 3, 7, 11, 15]"
4,u5,"[11, 7, 8, 12, 4, 13, 6]",[13]


In [ ]:
items_db = pd.DataFrame(
    {
        'item_id': [i+1 for i in range(item_range)],
        'item_price': [rn.randint(10,20000) for i in range(item_range)],
    }
)
items_db

,item_id,item_price
0,1,17237
1,2,11775
2,3,19969
3,4,13623
4,5,838
5,6,17630
6,7,9370
7,8,2580
8,9,9827
9,10,11723


In [ ]:
k = 4 # задать k
users = users_rec_bgt_db['user_id'].nunique()
# id = rn.randint(1, users-1) # для случайного пользователя
id = 1 # запрашиваем метрики по пользователю №1

user_id = 'u' + str(id)

# что рекомендовали пользователю
user_recs = users_rec_bgt_db.recommended_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# что купил пользователь
user_bgts = users_rec_bgt_db.bought_list.loc[users_rec_bgt_db['user_id'] == user_id].values[0]
# цены рекомендованных товаров
prices_list_rec = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_recs]
# цены реально купленных товаров
prices_list_bgt = [items_db.loc[items_db['item_id'] == item].item_price.values[0] for item in user_bgts]

# датафрейм
user = pd.DataFrame(
    {
        'user_id': user_id,
        'rec_items': [user_recs],
        'rel_items': [user_bgts],
        'rec_prices': [prices_list_rec],
        'rel_prices': [prices_list_bgt]
    }
)

# столбцы с метриками
# funcs1, cols1 -- ML-метрики без привязки топ-k, где funcs1 -- список функций, а cols1 -- имена столбцов
funcs1 = (hit_rate, precision, recall, mrr, dcg, ndcg)
cols1 = ('hr', 'p', 'r', 'mrr', 'dcg', 'ndcg')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs1, cols1):
  user[col] = user.apply(lambda user: func(user.rec_items, user.rel_items), axis=1)

# funcs2, cols2 -- ML-метрики на топ-k. они считаются отдельно, т.к. при их вызове нужно задать третий аргумент: k
funcs2 = (hit_rate_at_k,
          precision_at_k, average_precision_at_k,
          recall_at_k, average_recall_at_k,
          mrr_at_k, dcg_at_k, ndcg_at_k)
cols2 = ('hr@', 'p@', 'ap@', 'r@', 'ar@', 'mrr@', 'dcg@', 'ndcg@')
# рассчитать метрики через вызов функций и сформировать столбцы
for func, col in zip(funcs2, cols2):
#   print(func(user.rec_items, user.rel_items, k))
  try:
    print(func(user.rec_items, user.rel_items, k))
    user[col+str(k)] = user.apply(lambda user: func(user.rec_items, user.rel_items, k), axis=1)
  except IndexError:
    user[col+str(k)] = user.apply(lambda user: 0.5, axis=1)

# бизнес-метрики. nfr;t cчитаются отдельно, т.к. при их вызове нужно задать дополнительные аргумент: передать список/списки с ценами
user['mp@'+str(k)] = user.apply(lambda user: money_precision_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    k), axis=1)
user['mr@'+str(k)] = user.apply(lambda user: money_recall_at_k(
    user.rec_items,
    user.rel_items,
    user.rec_prices,
    user.rel_prices,
    k), axis=1)

# вывод
# user_file
print(user.iloc[0])

0
0
0
0
0.6666666666666666
0.0
0
0.0
user_id                                                    u1
rec_items                             [11, 1, 8, 10, 6, 3, 9]
rel_items                                          [8, 6, 10]
rec_prices    [12470, 17237, 2580, 11723, 17630, 19969, 9827]
rel_prices                               [2580, 17630, 11723]
hr                                                          1
p                                                    0.428571
r                                                         1.0
mrr                                                  0.261111
dcg                                                  1.317529
ndcg                                                 0.618289
hr@4                                                        1
p@4                                                       0.5
ap@4                                                 0.208333
r@4                                                  0.666667
ar@4                             